# 好奇心驱动探索 (Curiosity-Driven Exploration) 深度教程

## 目录
1. [问题背景：稀疏奖励与探索困境](#1-问题背景稀疏奖励与探索困境)
2. [内在动机理论基础](#2-内在动机理论基础)
3. [ICM: 内在好奇心模块](#3-icm-内在好奇心模块)
4. [RND: 随机网络蒸馏](#4-rnd-随机网络蒸馏)
5. [基于计数的探索](#5-基于计数的探索)
6. [实验对比与分析](#6-实验对比与分析)

## 1. 问题背景：稀疏奖励与探索困境

### 1.1 为什么需要内在动机？

**稀疏奖励环境的挑战**：
- 随机探索效率极低：$P(\text{success}) \approx (1/|A|)^T$
- 无法获得学习信号来改进策略
- 例如：Montezuma's Revenge 游戏需要数千步精确操作才能获得第一个奖励

### 1.2 内在动机的生物学启发

人类和动物具有**内在好奇心**：
- 婴儿探索环境不是为了外在奖励
- 科学家追求知识的本能驱动
- 新颖性本身就是一种奖励

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from typing import Tuple, List, Dict, Optional
import sys
sys.path.append('..')

np.random.seed(42)
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['font.size'] = 12
plt.rcParams['figure.figsize'] = (10, 6)

## 2. 内在动机理论基础

### 2.1 核心公式

总奖励由外在和内在奖励组成：

$$r_{total} = r_{extrinsic} + \beta \cdot r_{intrinsic}$$

其中 $\beta$ 控制探索与利用的平衡。

### 2.2 内在奖励的设计原则

1. **新颖性**：访问新状态应该获得奖励
2. **可学习性**：智能体能从探索中学到知识
3. **消退性**：重复访问同一状态的奖励应减少

### 2.3 内在奖励的分类

| 类型 | 信号来源 | 代表方法 |
|------|----------|----------|
| 预测误差 | 世界模型预测失败 | ICM, RND |
| 状态计数 | 访问次数 | Count-based |
| 信息增益 | 模型不确定性降低 | VIME |
| 能力获取 | 新技能学习 | Empowerment |

In [ ]:
def visualize_intrinsic_motivation():
    """可视化内在动机的直觉。"""
    
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))
    
    # 图1: 预测误差作为新颖性
    ax = axes[0]
    x = np.linspace(0, 100, 100)
    # 访问次数增加，预测误差降低
    prediction_error = 1.0 / (1 + 0.1 * x) + 0.1 * np.random.randn(100) * 0.1
    ax.plot(x, prediction_error, 'b-', linewidth=2)
    ax.fill_between(x, prediction_error, alpha=0.3)
    ax.set_xlabel('访问次数')
    ax.set_ylabel('预测误差 (内在奖励)')
    ax.set_title('预测误差随熟悉度下降')
    ax.grid(True, alpha=0.3)
    
    # 图2: 探索覆盖对比
    ax = axes[1]
    np.random.seed(42)
    
    # 随机探索（集中在起点附近）
    random_explore = np.random.randn(200, 2) * 0.5
    
    # 好奇心探索（更广泛分布）
    curious_explore = np.random.randn(200, 2) * 2.0
    
    ax.scatter(random_explore[:, 0], random_explore[:, 1], 
               c='blue', alpha=0.5, label='随机探索', s=20)
    ax.scatter(curious_explore[:, 0], curious_explore[:, 1],
               c='red', alpha=0.5, label='好奇心探索', s=20)
    ax.set_xlabel('状态维度1')
    ax.set_ylabel('状态维度2')
    ax.set_title('探索覆盖对比')
    ax.legend()
    ax.set_xlim(-5, 5)
    ax.set_ylim(-5, 5)
    
    # 图3: 学习曲线对比
    ax = axes[2]
    episodes = np.arange(500)
    
    # 无内在动机：学习缓慢
    no_intrinsic = 1 - np.exp(-episodes / 300)
    no_intrinsic += np.random.randn(500) * 0.05
    
    # 有内在动机：学习更快
    with_intrinsic = 1 - np.exp(-episodes / 100)
    with_intrinsic += np.random.randn(500) * 0.03
    
    ax.plot(episodes, np.clip(no_intrinsic, 0, 1), 'b-', 
            label='无内在动机', alpha=0.8)
    ax.plot(episodes, np.clip(with_intrinsic, 0, 1), 'r-',
            label='有内在动机', alpha=0.8)
    ax.set_xlabel('训练回合')
    ax.set_ylabel('成功率')
    ax.set_title('学习效率对比')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

visualize_intrinsic_motivation()

## 3. ICM: 内在好奇心模块

### 3.1 核心思想

ICM使用预测误差作为内在奖励：

$$r_i(s_t, a_t, s_{t+1}) = \eta \cdot \|f(s_{t+1}) - \hat{f}(s_t, a_t)\|^2$$

### 3.2 架构组成

```
观测 s ──→ [特征编码器 φ] ──→ f(s)
              │
              ├──→ [前向模型] ──→ f̂(s')
              │     ↑ 输入: f(s), a
              │     ↓ 输出: 预测的下一状态特征
              │
              └──→ [逆向模型] ──→ â
                    ↑ 输入: f(s), f(s')
                    ↓ 输出: 预测的动作
```

### 3.3 为什么在特征空间预测？

原始观测空间包含大量**不可控噪声**：
- 游戏中的云朵移动
- 树叶的随机摆动
- 与智能体动作无关的变化

特征空间只保留**动作相关信息**，由逆向模型强制约束。

In [ ]:
from curiosity_driven import (
    CuriosityConfig,
    FeatureEncoder,
    ForwardDynamicsModel,
    InverseDynamicsModel,
    IntrinsicCuriosityModule,
)

In [ ]:
# 创建ICM组件
obs_dim = 16
action_dim = 4
feature_dim = 32

# 特征编码器
encoder = FeatureEncoder(
    input_dim=obs_dim,
    feature_dim=feature_dim,
    hidden_dims=(64, 32),
)

# 前向动力学模型
forward_model = ForwardDynamicsModel(
    feature_dim=feature_dim,
    action_dim=action_dim,
    hidden_dims=(64, 32),
)

# 逆向动力学模型
inverse_model = InverseDynamicsModel(
    feature_dim=feature_dim,
    action_dim=action_dim,
    discrete_actions=True,
)

print("ICM组件创建成功！")
print(f"特征编码器: {obs_dim} → {feature_dim}")
print(f"前向模型: (特征{feature_dim}, 动作{action_dim}) → 特征{feature_dim}")
print(f"逆向模型: (特征{feature_dim}, 特征{feature_dim}) → 动作概率{action_dim}")

In [ ]:
# 演示ICM的工作流程
def demonstrate_icm_workflow():
    """展示ICM的完整工作流程。"""
    
    # 模拟一个转移
    state = np.random.randn(obs_dim)
    action = 2  # 选择动作2
    next_state = state + 0.1 * np.random.randn(obs_dim)  # 微小变化
    
    print("=" * 60)
    print("ICM工作流程演示")
    print("=" * 60)
    
    # 步骤1: 编码状态
    print("\n[步骤1] 特征编码")
    features_s = encoder.encode(state)
    features_s_next = encoder.encode(next_state)
    print(f"  状态特征 f(s): shape={features_s.shape}, norm={np.linalg.norm(features_s):.4f}")
    print(f"  下一状态特征 f(s'): shape={features_s_next.shape}")
    
    # 步骤2: 前向预测
    print("\n[步骤2] 前向模型预测")
    predicted_features = forward_model.predict(features_s, np.array([action]))
    print(f"  预测特征 f̂(s'): shape={predicted_features.shape}")
    
    # 步骤3: 计算预测误差（内在奖励）
    print("\n[步骤3] 计算预测误差")
    prediction_error = np.sum((features_s_next - predicted_features) ** 2)
    print(f"  预测误差 ||f(s') - f̂(s')||²: {prediction_error:.6f}")
    
    # 步骤4: 逆向模型预测动作
    print("\n[步骤4] 逆向模型预测动作")
    action_probs = inverse_model.predict(features_s, features_s_next)
    predicted_action = np.argmax(action_probs)
    print(f"  动作概率分布: {action_probs}")
    print(f"  真实动作: {action}, 预测动作: {predicted_action}")
    
    # 步骤5: 计算内在奖励
    print("\n[步骤5] 内在奖励")
    eta = 0.01  # 缩放系数
    intrinsic_reward = eta * prediction_error
    print(f"  r_i = η × ||f(s') - f̂(s')||² = {eta} × {prediction_error:.4f} = {intrinsic_reward:.6f}")
    
    return intrinsic_reward

intrinsic_r = demonstrate_icm_workflow()

In [ ]:
# 使用完整的ICM模块
config = CuriosityConfig(
    intrinsic_reward_scale=0.01,
    feature_dim=32,
    learning_rate=0.001,
    forward_loss_weight=0.2,
    inverse_loss_weight=0.8,
    normalize_rewards=True,
)

icm = IntrinsicCuriosityModule(
    observation_dim=obs_dim,
    action_dim=action_dim,
    config=config,
    discrete_actions=True,
)

print("ICM模块配置:")
print(f"  内在奖励缩放: {config.intrinsic_reward_scale}")
print(f"  特征维度: {config.feature_dim}")
print(f"  前向损失权重: {config.forward_loss_weight}")
print(f"  逆向损失权重: {config.inverse_loss_weight}")

In [ ]:
# 模拟ICM在训练中的行为
def simulate_icm_training(icm, n_episodes=50, episode_length=100):
    """模拟ICM训练过程。"""
    
    history = {
        'intrinsic_rewards': [],
        'forward_losses': [],
        'inverse_losses': [],
    }
    
    for ep in range(n_episodes):
        state = np.random.randn(obs_dim)
        ep_intrinsic = []
        
        for _ in range(episode_length):
            action = np.array([np.random.randint(0, action_dim)])
            next_state = state + 0.1 * np.random.randn(obs_dim)
            
            # 计算内在奖励
            intrinsic = icm.compute_intrinsic_reward(state, action, next_state)
            ep_intrinsic.append(intrinsic)
            
            # 更新模型
            stats = icm.update(state, action, next_state)
            
            state = next_state
        
        history['intrinsic_rewards'].append(np.mean(ep_intrinsic))
        history['forward_losses'].append(stats['forward_loss'])
        history['inverse_losses'].append(stats['inverse_loss'])
    
    return history

print("开始ICM训练模拟...")
icm_history = simulate_icm_training(icm)
print("训练完成！")

In [ ]:
# 可视化ICM训练过程
def plot_icm_training(history):
    """绘制ICM训练曲线。"""
    
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))
    
    # 内在奖励
    axes[0].plot(history['intrinsic_rewards'], 'b-', alpha=0.8)
    axes[0].set_xlabel('回合')
    axes[0].set_ylabel('平均内在奖励')
    axes[0].set_title('内在奖励变化')
    axes[0].grid(True, alpha=0.3)
    
    # 前向损失
    axes[1].plot(history['forward_losses'], 'g-', alpha=0.8)
    axes[1].set_xlabel('回合')
    axes[1].set_ylabel('前向模型损失')
    axes[1].set_title('前向预测损失')
    axes[1].grid(True, alpha=0.3)
    
    # 逆向损失
    axes[2].plot(history['inverse_losses'], 'r-', alpha=0.8)
    axes[2].set_xlabel('回合')
    axes[2].set_ylabel('逆向模型损失')
    axes[2].set_title('逆向预测损失')
    axes[2].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("观察结果:")
    print("  - 内在奖励随训练下降 → 模型学会了预测熟悉的状态")
    print("  - 前向损失下降 → 世界模型在改进")
    print("  - 逆向损失下降 → 特征编码器在学习动作相关信息")

plot_icm_training(icm_history)

## 4. RND: 随机网络蒸馏

### 4.1 核心思想

RND使用一个**固定的随机网络**作为目标，训练预测网络去匹配它：

$$r_i(s) = \|f_{rand}(s) - \hat{f}(s)\|^2$$

### 4.2 为什么有效？

1. 预测网络只在**访问过的状态**上训练
2. 新状态 → 预测误差高 → 高内在奖励
3. 旧状态 → 预测误差低 → 低内在奖励

### 4.3 RND vs ICM

| 特性 | ICM | RND |
|------|-----|-----|
| 目标网络 | 学习的特征 | 固定随机 |
| 逆向模型 | 需要 | 不需要 |
| 实现复杂度 | 较高 | 较低 |
| 随机噪声鲁棒性 | 较好 | 较差 |

In [ ]:
from curiosity_driven import RandomNetworkDistillation

# 创建RND
rnd = RandomNetworkDistillation(
    observation_dim=obs_dim,
    feature_dim=32,
    hidden_dims=(64, 32),
    learning_rate=0.001,
    intrinsic_reward_scale=0.01,
)

print("RND模块创建成功！")

In [ ]:
# 演示RND的核心特性：新状态有高奖励
def demonstrate_rnd_novelty():
    """演示RND对新颖性的响应。"""
    
    # 创建一个"熟悉"的状态区域
    familiar_states = np.random.randn(100, obs_dim) * 0.5  # 集中在原点附近
    
    # 训练RND在熟悉状态上
    print("训练RND在熟悉状态区域...")
    for _ in range(200):
        for state in familiar_states:
            rnd.update(state)
    
    # 测试不同区域的内在奖励
    test_regions = {
        '熟悉区域 (原点附近)': np.random.randn(20, obs_dim) * 0.5,
        '边缘区域': np.random.randn(20, obs_dim) * 2.0,
        '远离区域': np.random.randn(20, obs_dim) * 5.0,
    }
    
    results = {}
    for name, states in test_regions.items():
        rewards = [rnd.compute_intrinsic_reward(s) for s in states]
        results[name] = np.mean(rewards)
        print(f"  {name}: 平均内在奖励 = {results[name]:.6f}")
    
    # 可视化
    plt.figure(figsize=(8, 5))
    bars = plt.bar(results.keys(), results.values(), color=['green', 'orange', 'red'])
    plt.ylabel('平均内在奖励')
    plt.title('RND: 新颖状态获得更高奖励')
    plt.xticks(rotation=15)
    plt.grid(True, alpha=0.3, axis='y')
    plt.tight_layout()
    plt.show()
    
    print("\n关键洞察: 远离训练分布的状态获得更高的内在奖励！")

demonstrate_rnd_novelty()

## 5. 基于计数的探索

### 5.1 核心公式

$$r_i(s) = \frac{\beta}{\sqrt{N(s)}}$$

其中 $N(s)$ 是状态 $s$ 的访问次数。

### 5.2 优缺点

**优点**：
- 理论基础扎实（UCB bounds）
- 计算简单 O(1)
- 对随机环境鲁棒

**缺点**：
- 需要状态离散化
- 连续空间维度灾难

In [ ]:
from curiosity_driven import CountBasedExploration

# 创建计数探索器
counter = CountBasedExploration(
    state_discretization=10,  # 每个维度10个bin
    bonus_coefficient=1.0,
    intrinsic_reward_scale=0.1,
)

# 模拟探索过程
print("模拟基于计数的探索...\n")

# 重复访问同一区域
repeated_state = np.array([0.5, 0.5])
rewards_over_visits = []

for i in range(10):
    reward = counter.compute_intrinsic_reward(repeated_state)
    rewards_over_visits.append(reward)
    counter.update(repeated_state)
    print(f"访问 {i+1}: N(s)={counter.get_count(repeated_state)}, r_i={reward:.4f}")

# 可视化
plt.figure(figsize=(8, 4))
plt.plot(range(1, 11), rewards_over_visits, 'bo-', markersize=8)
plt.xlabel('访问次数')
plt.ylabel('内在奖励')
plt.title('基于计数的探索: 重复访问奖励递减')
plt.grid(True, alpha=0.3)
plt.show()

## 6. 实验对比与分析

### 6.1 不同方法在GridWorld上的表现

In [ ]:
# 简单的GridWorld环境
class SimpleGridWorld:
    """简单的GridWorld用于测试探索算法。"""
    
    def __init__(self, size=10):
        self.size = size
        self.state = np.array([0, 0])
        self.goal = np.array([size-1, size-1])
        
    def reset(self):
        self.state = np.array([0, 0])
        return self._get_obs()
    
    def _get_obs(self):
        # 返回one-hot编码的位置
        obs = np.zeros(self.size * self.size)
        idx = self.state[0] * self.size + self.state[1]
        obs[idx] = 1.0
        return obs
    
    def step(self, action):
        # 动作: 0=上, 1=下, 2=左, 3=右
        moves = [[-1, 0], [1, 0], [0, -1], [0, 1]]
        new_state = self.state + np.array(moves[action])
        new_state = np.clip(new_state, 0, self.size - 1)
        self.state = new_state
        
        done = np.array_equal(self.state, self.goal)
        reward = 1.0 if done else 0.0  # 稀疏奖励
        
        return self._get_obs(), reward, done, {}

# 测试环境
env = SimpleGridWorld(size=5)
print(f"环境状态空间大小: {env.size * env.size}")
print(f"目标位置: {env.goal}")

In [ ]:
def compare_exploration_methods(env, n_episodes=100, max_steps=50):
    """对比不同探索方法的覆盖率。"""
    
    obs_dim = env.size * env.size
    action_dim = 4
    
    # 方法1: 随机探索
    random_coverage = set()
    
    # 方法2: ICM探索
    icm = IntrinsicCuriosityModule(
        observation_dim=obs_dim,
        action_dim=action_dim,
        config=CuriosityConfig(intrinsic_reward_scale=0.1),
    )
    icm_coverage = set()
    
    # 方法3: 计数探索
    counter = CountBasedExploration(
        state_discretization=env.size,
        intrinsic_reward_scale=0.1,
    )
    count_coverage = set()
    
    # 运行实验
    for ep in range(n_episodes):
        # 随机探索
        state = env.reset()
        for _ in range(max_steps):
            action = np.random.randint(0, action_dim)
            next_state, _, done, _ = env.step(action)
            random_coverage.add(tuple(env.state))
            state = next_state
            if done:
                break
        
        # ICM探索（偏向高内在奖励的动作）
        state = env.reset()
        for _ in range(max_steps):
            # 简化：随机动作但更新ICM
            action = np.random.randint(0, action_dim)
            next_state, _, done, _ = env.step(action)
            icm.compute_intrinsic_reward(state, np.array([action]), next_state)
            icm_coverage.add(tuple(env.state))
            state = next_state
            if done:
                break
        
        # 计数探索
        state = env.reset()
        for _ in range(max_steps):
            action = np.random.randint(0, action_dim)
            next_state, _, done, _ = env.step(action)
            counter.update(env.state.astype(float) / env.size)  # 归一化
            count_coverage.add(tuple(env.state))
            state = next_state
            if done:
                break
    
    total_states = env.size * env.size
    results = {
        '随机探索': len(random_coverage) / total_states,
        'ICM探索': len(icm_coverage) / total_states,
        '计数探索': len(count_coverage) / total_states,
    }
    
    return results

print("运行探索方法对比实验...")
coverage_results = compare_exploration_methods(env)

# 可视化结果
plt.figure(figsize=(8, 5))
bars = plt.bar(coverage_results.keys(), coverage_results.values(),
               color=['blue', 'green', 'orange'])
plt.ylabel('状态覆盖率')
plt.title('不同探索方法的状态空间覆盖率对比')
plt.ylim(0, 1.1)

for bar, val in zip(bars, coverage_results.values()):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02,
             f'{val:.1%}', ha='center')

plt.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

## 总结

### 核心要点

1. **内在动机**解决稀疏奖励问题，通过生成额外的学习信号

2. **ICM**使用预测误差作为新颖性度量，逆向模型确保特征与动作相关

3. **RND**更简单，使用固定随机网络作为目标，但对随机噪声敏感

4. **计数探索**理论基础扎实，但需要状态离散化

### 方法选择指南

| 场景 | 推荐方法 |
|------|----------|
| 视觉观测（图像）| ICM |
| 低维状态空间 | 计数探索 |
| 快速原型验证 | RND |
| 随机环境 | ICM 或 集成方法 |

In [ ]:
# 打印总结
summary = """
╔══════════════════════════════════════════════════════════════════════╗
║                    好奇心驱动探索 - 核心总结                         ║
╠══════════════════════════════════════════════════════════════════════╣
║                                                                      ║
║  内在奖励公式：                                                      ║
║    r_total = r_extrinsic + β · r_intrinsic                          ║
║                                                                      ║
║  ICM内在奖励：                                                       ║
║    r_i = η · ||f(s') - f̂(s, a)||²                                  ║
║                                                                      ║
║  RND内在奖励：                                                       ║
║    r_i = ||f_rand(s) - f̂(s)||²                                     ║
║                                                                      ║
║  计数探索奖励：                                                      ║
║    r_i = β / √N(s)                                                  ║
║                                                                      ║
╚══════════════════════════════════════════════════════════════════════╝
"""
print(summary)